# Caso de Negocio

### [BBT3-9](https://alkemy-labs.atlassian.net/browse/BBT3-9) Fabricio

### [BBT3-10](https://alkemy-labs.atlassian.net/browse/BBT3-10) Judith

### [BBT3-11](https://alkemy-labs.atlassian.net/browse/BBT3-11) Pablo

### Creacion notebook e importación de librerías.

In [1]:
! pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285387 sha256=dd1202d3034853b39a15ebddfebc4b704e7802845f08ecd02e8a2665cebb44b9
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [2]:
import pandas as pd
from pyspark.sql import SparkSession
import sqlite3 as sql

## Datasets a base de datos

### [BBT3-12](https://alkemy-labs.atlassian.net/browse/BBT3-12) Fabricio

### [BBT3-13](https://alkemy-labs.atlassian.net/browse/BBT3-13) Judith

### [BBT3-14](https://alkemy-labs.atlassian.net/browse/BBT3-14) Pablo

In [3]:
order_items_link_csv = 'https://raw.githubusercontent.com/alkemyTech/BBVA-BDS-W1-T3/main/datasets/Copia%20de%20ecommerce_order_items_dataset.csv'
customers_link_csv = 'https://raw.githubusercontent.com/alkemyTech/BBVA-BDS-W1-T3/main/datasets/Copia%20de%20ecommerce_customers_dataset.csv'
geolocation_link_csv = 'https://raw.githubusercontent.com/alkemyTech/BBVA-BDS-W1-T3/main/datasets/Copia%20de%20ecommerce_geolocation_dataset.csv'
sellers_link_csv = 'https://raw.githubusercontent.com/alkemyTech/BBVA-BDS-W1-T3/main/datasets/Copia%20de%20ecommerce_sellers_dataset.csv'
order_payments_link_csv = 'https://raw.githubusercontent.com/alkemyTech/BBVA-BDS-W1-T3/main/datasets/Copia%20de%20ecommerce_order_payments_dataset.csv'
product_category_name_translation_link_csv = 'https://raw.githubusercontent.com/alkemyTech/BBVA-BDS-W1-T3/main/datasets/Copia%20de%20product_category_name_translation.csv'
order_reviews_link_csv = 'https://raw.githubusercontent.com/alkemyTech/BBVA-BDS-W1-T3/main/datasets/Copia%20de%20ecommerce_order_reviews_dataset.csv'
orders_link_csv = 'https://github.com/alkemyTech/BBVA-BDS-W1-T3/raw/main/datasets/Copia%20de%20ecommerce_orders_dataset.csv'
products_link_csv = 'https://raw.githubusercontent.com/alkemyTech/BBVA-BDS-W1-T3/main/datasets/Copia%20de%20ecommerce_products_dataset.csv'


order_items_df = pd.read_csv(order_items_link_csv)
order_payments_df = pd.read_csv(order_payments_link_csv)
products_df = pd.read_csv(products_link_csv)
customers_df = pd.read_csv(customers_link_csv)
orders_df = pd.read_csv(orders_link_csv)
order_reviews_df = pd.read_csv(order_reviews_link_csv)
geolocation_df = pd.read_csv(geolocation_link_csv)
sellers_df = pd.read_csv(sellers_link_csv)
product_category_name_translation_df = pd.read_csv(product_category_name_translation_link_csv)

In [4]:
# seteo de columnas indices:
order_items_df.set_index(['order_id', 'order_item_id'], inplace=True)
order_payments_df.set_index(['order_id','payment_sequential'], inplace=True)
products_df.set_index('product_id', inplace=True)
customers_df.set_index('customer_id', inplace=True)
orders_df.set_index('order_id', inplace=True)
order_reviews_df.set_index('review_id', inplace=True)
geolocation_df.set_index('geolocation_zip_code_prefix', inplace=True)
sellers_df.set_index('seller_id', inplace=True)
product_category_name_translation_df.set_index('product_category_name', inplace=True)

In [5]:
# creacion base de datos
conn = sql.connect('ecommerce.db')
cursor = conn.cursor()

In [6]:
# guardado de dataframes en tablas de la base de datos
order_payments_df.to_sql('order_payments', con=conn, if_exists='replace')
order_items_df.to_sql('order_items', con=conn, if_exists='replace')
orders_df.to_sql('orders', con=conn, if_exists='replace')
customers_df.to_sql('customers', con=conn, if_exists='replace')
products_df.to_sql('products', con=conn, if_exists='replace')
order_reviews_df.to_sql('order_reviews', con=conn, if_exists='replace')
geolocation_df.to_sql('geolocation', con=conn, if_exists='replace')
sellers_df.to_sql('sellers', con=conn, if_exists='replace')
product_category_name_translation_df.to_sql('product_category_name_translation', con=conn, if_exists='replace')

71

In [7]:
# query para obtener los nombres de las tablas creadas
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
lista_tablas = cursor.fetchall()
lista_nombre_tablas = [tabla[0] for tabla in lista_tablas]
print(lista_nombre_tablas)

['order_payments', 'order_items', 'orders', 'customers', 'products', 'order_reviews', 'geolocation', 'sellers', 'product_category_name_translation']


## Extraccion de Base de Datos


### orders_items y sellers
###[BBT3-17](https://alkemy-labs.atlassian.net/browse/BBT3-17) Judith

In [8]:
query_items_seller = '''
  SELECT * FROM order_items
  INNER JOIN sellers
  ON order_items.seller_id = sellers.seller_id
'''
items_seller_df = pd.read_sql_query(query_items_seller, conn)
items_seller_df

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,48436dade18ac8b2bce089ec2a041202,27277,volta redonda,SP
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93,dd7ddc04e1b6c2c614352b383efe2d36,3471,sao paulo,SP
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87,5b51032eddd242adc84c38acab88f23d,37564,borda da mata,MG
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79,9d7a1d34a5052409006425275ba1c2b4,14403,franca,SP
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14,df560393f3a51e74553ab94004ba5c87,87900,loanda,PR
...,...,...,...,...,...,...,...,...,...,...,...
112645,fffc94f6ce00a00581880bf54a75a037,1,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-05-02 04:11:01,299.99,43.41,b8bc237ba3788b23da09c0f1f3a3288c,88303,itajai,SC
112646,fffcd46ef2263f404302a634eb57f7eb,1,32e07fd915822b0765e448c4dd74c828,f3c38ab652836d21de61fb8314b69182,2018-07-20 04:31:48,350.00,36.53,f3c38ab652836d21de61fb8314b69182,1206,sao paulo,SP
112647,fffce4705a9662cd70adb13d4a31832d,1,72a30483855e2eafc67aee5dc2560482,c3cfdc648177fdbbbb35635a37472c53,2017-10-30 17:14:25,99.90,16.95,c3cfdc648177fdbbbb35635a37472c53,80610,curitiba,PR
112648,fffe18544ffabc95dfada21779c9644f,1,9c422a519119dcad7575db5af1ba540e,2b3e4a2a3ea8e01938cabda2a3e5cc79,2017-08-21 00:04:32,55.99,8.72,2b3e4a2a3ea8e01938cabda2a3e5cc79,4733,sao paulo,SP


### orders_items y products
### [BBT3-18](https://alkemy-labs.atlassian.net/browse/BBT3-18) Judith

In [9]:
query_items_products = '''
  SELECT * FROM order_items
  INNER JOIN products
  ON order_items.product_id = products.product_id
'''
items_products_df = pd.read_sql_query(query_items_products, conn)
items_products_df

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,4244733e06e7ecb4970a6e2683c13e61,cool_stuff,58.0,598.0,4.0,650.0,28.0,9.0,14.0
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93,e5f2d52b802189ee658865ca93d83a8f,pet_shop,56.0,239.0,2.0,30000.0,50.0,30.0,40.0
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87,c777355d18b72b67abbeef9df44fd0fd,moveis_decoracao,59.0,695.0,2.0,3050.0,33.0,13.0,33.0
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79,7634da152a4610f1595efa32f14722fc,perfumaria,42.0,480.0,1.0,200.0,16.0,10.0,15.0
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14,ac6c3623068f30de03045865e4e10089,ferramentas_jardim,59.0,409.0,1.0,3750.0,35.0,40.0,30.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112645,fffc94f6ce00a00581880bf54a75a037,1,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-05-02 04:11:01,299.99,43.41,4aa6014eceb682077f9dc4bffebc05b0,utilidades_domesticas,43.0,1002.0,3.0,10150.0,89.0,15.0,40.0
112646,fffcd46ef2263f404302a634eb57f7eb,1,32e07fd915822b0765e448c4dd74c828,f3c38ab652836d21de61fb8314b69182,2018-07-20 04:31:48,350.00,36.53,32e07fd915822b0765e448c4dd74c828,informatica_acessorios,31.0,232.0,1.0,8950.0,45.0,26.0,38.0
112647,fffce4705a9662cd70adb13d4a31832d,1,72a30483855e2eafc67aee5dc2560482,c3cfdc648177fdbbbb35635a37472c53,2017-10-30 17:14:25,99.90,16.95,72a30483855e2eafc67aee5dc2560482,esporte_lazer,43.0,869.0,1.0,967.0,21.0,24.0,19.0
112648,fffe18544ffabc95dfada21779c9644f,1,9c422a519119dcad7575db5af1ba540e,2b3e4a2a3ea8e01938cabda2a3e5cc79,2017-08-21 00:04:32,55.99,8.72,9c422a519119dcad7575db5af1ba540e,informatica_acessorios,56.0,1306.0,1.0,100.0,20.0,20.0,20.0


### orders y order_payments
### [BBT3-19](https://alkemy-labs.atlassian.net/browse/BBT3-19) Judith

In [10]:
query_order_payments = '''
  SELECT * FROM orders
  INNER JOIN order_payments
  ON orders.order_id = order_payments.order_id
'''
query_order_payments = pd.read_sql_query(query_order_payments, conn)
query_order_payments

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,0a8556ac6be836b46b3e89920d59291c,delivered,2018-04-25 22:01:49,2018-04-25 22:15:09,2018-05-02 15:20:00,2018-05-09 17:36:51,2018-05-22 00:00:00,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,f2c7fc58a9de810828715166c672f10a,delivered,2018-06-26 11:01:38,2018-06-26 11:18:58,2018-06-28 14:18:00,2018-06-29 20:32:09,2018-07-16 00:00:00,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,25b14b69de0b6e184ae6fe2755e478f9,delivered,2017-12-12 11:19:55,2017-12-14 09:52:34,2017-12-15 20:13:22,2017-12-18 17:24:41,2018-01-04 00:00:00,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,7a5d8efaaa1081f800628c30d2b0728f,delivered,2017-12-06 12:04:06,2017-12-06 12:13:20,2017-12-07 20:28:28,2017-12-21 01:35:51,2018-01-04 00:00:00,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,15fd6fb8f8312dbb4674e4518d6fa3b3,delivered,2018-05-21 13:59:17,2018-05-21 16:14:41,2018-05-22 11:46:00,2018-06-01 21:44:53,2018-06-13 00:00:00,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45
...,...,...,...,...,...,...,...,...,...,...,...,...,...
103881,0406037ad97740d563a178ecc7a2075c,5d576cb2dfa3bc05612c392a1ee9c654,delivered,2018-03-08 16:57:23,2018-03-10 03:55:25,2018-03-12 18:19:36,2018-03-16 13:09:51,2018-04-04 00:00:00,0406037ad97740d563a178ecc7a2075c,1,boleto,1,363.31
103882,7b905861d7c825891d6347454ea7863f,2079230c765a88530822a34a4cec2aa0,delivered,2017-08-18 09:45:35,2017-08-18 10:04:56,2017-08-18 18:04:24,2017-08-23 22:25:56,2017-09-12 00:00:00,7b905861d7c825891d6347454ea7863f,1,credit_card,2,96.80
103883,32609bbb3dd69b3c066a6860554a77bf,e4abb5057ec8cfda9759c0dc415a8188,invoiced,2017-11-18 17:27:14,2017-11-18 17:46:05,None,None,2017-12-05 00:00:00,32609bbb3dd69b3c066a6860554a77bf,1,credit_card,1,47.77
103884,b8b61059626efa996a60be9bb9320e10,5d719b0d300663188169c6560e243f27,delivered,2018-08-07 23:26:13,2018-08-07 23:45:00,2018-08-09 11:46:00,2018-08-21 22:41:46,2018-08-24 00:00:00,b8b61059626efa996a60be9bb9320e10,1,credit_card,5,369.54
